# Assignment parallel computing group 3

In [28]:
import fake_enformer 
fake_enformer.predict('hg38:chr4:8848348:A:T')
#  -0.0007830759576305497

-0.0007830759576305497

In [33]:
fake_enformer.predict('hg38:chr4:8848348:A:T')

-0.0007830759576305497

In [ ]:
!cat /lustre1/project/stg_00079/teaching/data/snps.annotated.vcf

## Import libraries

In [14]:
import vcfpy
import fake_enformer 
import pandas as pd
from multiprocessing import Pool
import multiprocessing

## Configuration

In [35]:
genome = "hg38"
vcf_path = '/lustre1/project/stg_00079/teaching/data/snps.annotated.vcf'

In [36]:
reader = vcfpy.Reader.from_path(vcf_path)

In [37]:
def process_variant(variant_str):
    """Wrapper function for prediction"""
    return fake_enformer.predict(variant_str)

### Generate all variant strings

In [38]:
variant_strings = []
records = []

for i, record in enumerate(reader):
    assert len(record.ALT) == 1
    chrom = record.CHROM
    pos = record.POS
    ref = record.REF
    alt = record.ALT[0].value

    variant_str = f"{genome}:{chrom}:{pos}:{ref}:{alt}"
    variant_strings.append(variant_str)
    records.append(record)

    

In [ ]:
#variant_strings

## Multiprocessing

In [24]:
# Create process pool
num_processes = multiprocessing.cpu_count()
num_processes

72

In [ ]:
with Pool(processes=num_processes,) as pool:
    results = pool.map(fake_enformer.predict, variant_strings, chunksize=100)

# Process results as needed
print(f"Processed {len(results)} variants")

In [32]:
results = map(fake_enformer.predict, variant_strings)
print(list(results))

[]


In [27]:
for val in variant_strings:
    fake_enformer.predict(value)


## Save to new VCF

In [ ]:
new_header = reader.header.copy()
new_header.add_info_line({
    'ID': 'EF',
    'Number': '1',
    'Type': 'Float',
    'Description': 'Enformer prediction score'
})

In [ ]:
with vcfpy.Writer.from_path("snps_annotated_new.vcf, new_header) as writer:
    for record, score in zip(records, results):
        if score is not None:
            # Add prediction to INFO field
            record.INFO['EF'] = score
        writer.write_record(record)

print(f"Successfully wrote {len(records)} variants to {output_vcf_path}")